```

#  INFO:     2405:4802:182d:20c0:4480:65c0:d6dd:cac4:0 - "GET /rag/nttu?q=xin%20ch%C3%A0o HTTP/1.1" 500 Internal Server Error
# <ipython-input-7-d0ef9f03339d>:38: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
#   db = Qdrant(client=client,
# <ipython-input-15-a883cb622b5a>:42: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
#   data = app.rag(source=source)(q)
```

```python
    try:
        # Thực hiện truy vấn RAG
        # data = app.rag(source=source)(q)
        data = app.rag(source=source).invoke(q)
```



1. **Test endpoint root**:
```bash
curl https://briefly-knowing-treefrog.ngrok-free.app/
```
=> oke
2. **Test endpoint RAG với source "nttu"**:
```bash
curl "https://briefly-knowing-treefrog.ngrok-free.app/rag/nttu?q=xin%20chao"
```
=> bug


```
{
    "detail": "An error occurred: 'VectorStoreRetriever' object has no attribute 'embeddings'"
}

Là lỗi gì 
```



Có, trong code của bạn đã tạo embeddings. Hãy phân tích từng phần:

1. **Tạo embeddings model**:
```python
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key = HUGGINGFACE_API_KEY,
    model_kwargs = {'device': 'auto'}
)
```
Đây là bước khởi tạo model embeddings sử dụng HuggingFace API.

2. **Sử dụng embeddings để tạo vector database**:
```python
qdrant = Qdrant.from_documents(
    texts_sotay,
    embeddings,  # Embeddings model được sử dụng ở đây
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="nttu_sotay_vector_db_v1",
    api_key=QDRANT_API_KEY,
)
```

3. **Kiểm tra embeddings hoạt động**:
```python
# Test tìm văn bản liên quan đến query
query = "Xếp loại học bổng ở trường"
found_docs = qdrant.similarity_search(query,k=15)
```
Nếu `similarity_search` hoạt động, điều này xác nhận rằng:
- Embeddings đã được tạo thành công
- Dữ liệu đã được lưu trong Qdrant database
- Quá trình tìm kiếm vector similarity đang hoạt động

Để kiểm tra thêm, bạn có thể:
```python
# Kiểm tra xem embeddings có hoạt động không
test_embedding = embeddings.embed_query("test query")
print(f"Embedding dimension: {len(test_embedding)}")
```

Nếu code trên chạy mà không có lỗi và trả về một vector có chiều dài hợp lý, thì embeddings đã được tạo và hoạt động đúng.


# Có những phần nào lưu vào database??
Những trường nào ?

Chưa hiểu lắm cách lưu Qdant trong code 



Hãy phân tích chi tiết cách dữ liệu được ghi vào Qdrant:

1. **Quá trình xử lý văn bản**:
```python
# 1. Load văn bản từ file
loader_sotay = TextLoader("sotay.txt")
documents_sotay = loader_sotay.load()

# 2. Cắt văn bản thành các đoạn nhỏ
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,    # Mỗi đoạn dài 800 ký tự
    chunk_overlap=400  # Overlap 400 ký tự để tránh mất ngữ cảnh
)
texts_sotay = text_splitter.split_documents(documents_sotay)
```

2. **Tạo embeddings**:
```python
# Khởi tạo model embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key = HUGGINGFACE_API_KEY,
    model_kwargs = {'device': 'auto'}
)
```

3. **Ghi vào Qdrant**:
```python
# Phương thức from_documents sẽ:
qdrant = Qdrant.from_documents(
    texts_sotay,      # List các document đã cắt nhỏ
    embeddings,       # Model để chuyển text thành vector
    url=QDRANT_URL,   # URL của Qdrant server
    collection_name="nttu_sotay_vector_db_v1",  # Tên collection
    api_key=QDRANT_API_KEY,
)
```

4. **Chi tiết quá trình ghi**:
```python
# Bên trong from_documents, quá trình diễn ra như sau:
for doc in texts_sotay:
    # 1. Tạo vector từ nội dung văn bản
    vector = embeddings.embed_query(doc.page_content)
    
    # 2. Tạo payload (metadata)
    payload = {
        "page_content": doc.page_content,
        "metadata": doc.metadata
    }
    
    # 3. Ghi vào Qdrant
    # - Vector được lưu để tìm kiếm similarity
    # - Payload được lưu kèm để trả về kết quả
```

5. **Kiểm tra dữ liệu đã lưu**:
```python
from qdrant_client import QdrantClient

# Kết nối tới Qdrant
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# Lấy thông tin collection
collection_info = client.get_collection("nttu_sotay_vector_db_v1")
print("Collection info:", collection_info)

# Lấy một số points để kiểm tra
points = client.scroll(
    collection_name="nttu_sotay_vector_db_v1",
    limit=2  # Lấy 2 điểm đầu tiên
)
for point in points[0]:
    print("\nPoint ID:", point.id)
    print("Payload:", point.payload)
```

Khi tìm kiếm:
```python
# Khi bạn search:
query = "Xếp loại học bổng ở trường"
found_docs = qdrant.similarity_search(query, k=15)

# Quá trình xảy ra:
# 1. Query được chuyển thành vector
# 2. Tìm k vectors gần nhất trong database
# 3. Trả về documents tương ứng với các vectors đó
```

Bạn có muốn tôi giải thích thêm phần nào không?


# QUAY LẠI BUG 
```
curl --location 'https://briefly-knowing-treefrog.ngrok-free.app/rag/nttu?q=xin%20chao'

{
    "detail": "An error occurred: 'VectorStoreRetriever' object has no attribute 'embeddings'"
}

bug này là sao
```

Lỗi `"An error occurred: 'VectorStoreRetriever' object has no attribute 'embeddings'"` cho thấy rằng trong mã của bạn, một đối tượng thuộc lớp `VectorStoreRetriever` đang cố gắng truy cập thuộc tính `embeddings`, nhưng thuộc tính này không tồn tại.

### Nguyên nhân và Giải pháp

1. **Kiểm tra Class `RerankRetriever`**:
   - Đảm bảo rằng bạn đã định nghĩa thuộc tính `embeddings` trong class `RerankRetriever`. Nếu không, bạn cần thêm nó.

   ```python
   class RerankRetriever(VectorStoreRetriever):
       vectorstore: VectorStoreRetriever

       @property
       def embeddings(self):
           return self.vectorstore.embeddings  # Đảm bảo vectorstore có embeddings
   ```

2. **Kiểm tra Khởi tạo `vectorstore`**:
   - Đảm bảo rằng `vectorstore` được khởi tạo với một đối tượng có thuộc tính `embeddings`. Nếu bạn đang sử dụng Qdrant, hãy chắc chắn rằng bạn đã khởi tạo đúng cách.

3. **Cập nhật `from_documents`**:
   - Nếu bạn đã cập nhật `langchain`, hãy kiểm tra xem có thay đổi nào trong cách khởi tạo `VectorStoreRetriever` không.

4. **Kiểm tra Mã Gọi API**:
   - Đảm bảo rằng mã gọi API đang sử dụng đúng class và không có lỗi trong quá trình khởi tạo.

### Ví dụ Cụ Thể

Dưới đây là một ví dụ về cách bạn có thể định nghĩa class `RerankRetriever`:

```python
class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever

    @property
    def embeddings(self):
        return self.vectorstore.embeddings  # Đảm bảo vectorstore có embeddings

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query] * len(candidates)
        features = tokenizer_rerank(queries, candidates, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
        return [docs[indices[0]], docs[indices[1]]]
```

### Kiểm Tra

- Sau khi thực hiện các thay đổi, hãy thử lại yêu cầu curl và xem lỗi có còn không.
- Nếu vẫn gặp lỗi, hãy kiểm tra log chi tiết của backend để biết thêm thông tin về nguyên nhân.


```python
from langchain.schema.document import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.retrievers import WikipediaRetriever
from typing import List
class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever
    @property
    def embeddings(self):
        return self.vectorstore.embeddings  # Đảm bảo vectorstore có embeddings
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]
class RerankWikiRetriever(VectorStoreRetriever):
    vectorstore: WikipediaRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]
```
<ipython-input-29-e5a01d4977d7>:5: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class RerankRetriever(VectorStoreRetriever):
<ipython-input-29-e5a01d4977d7>:20: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class RerankWikiRetriever(VectorStoreRetriever):



Cảnh báo `DeprecationWarning` cho biết rằng các lớp `RerankRetriever` và `RerankWikiRetriever` của bạn cần phải triển khai phương thức `_get_relevant_documents` thay vì `get_relevant_documents`. Điều này có nghĩa là API của `langchain` đã thay đổi và bạn cần cập nhật mã của mình để phù hợp với các thay đổi này.

### Cách Sửa

1. **Thay đổi tên phương thức**:
   - Thay thế `get_relevant_documents` bằng `_get_relevant_documents`.

2. **Cập nhật mã**:
   Dưới đây là cách sửa mã của bạn:

```python
from langchain.schema.document import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.retrievers import WikipediaRetriever
from typing import List

class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever

    @property
    def embeddings(self):
        return self.vectorstore.embeddings  # Đảm bảo vectorstore có embeddings

    def _get_relevant_documents(self, query: str) -> List[Document]:  # Thay đổi ở đây
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query] * len(candidates)
        features = tokenizer_rerank(queries, candidates, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
        return [docs[indices[0]], docs[indices[1]]]

class RerankWikiRetriever(VectorStoreRetriever):
    vectorstore: WikipediaRetriever

    def _get_relevant_documents(self, query: str) -> List[Document]:  # Thay đổi ở đây
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query] * len(candidates)
        features = tokenizer_rerank(queries, candidates, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
        return [docs[indices[0]], docs[indices[1]]]
```

### Giải thích
- **Phương thức `_get_relevant_documents`**: Đây là phương thức mới mà bạn cần triển khai để thay thế cho `get_relevant_documents`. Điều này giúp đảm bảo rằng mã của bạn tương thích với phiên bản mới của `langchain`.

### Kiểm tra lại
Sau khi thực hiện các thay đổi này, hãy chạy lại mã của bạn và kiểm tra xem cảnh báo có còn không. Nếu bạn gặp phải lỗi khác, hãy cho tôi biết để tôi có thể hỗ trợ thêm!


## Sau khi thực hiện kiểm tra các kiểu: thư viện cũ nên ko nhận được embedding cần update code, .. => Mình phát hiện ra bug có thể ở data: check data thì thấy: DATASET content bị LƯU CHUNKING SAI SAI. 


```
page_content
a=this.i.length,b=this.i,c=[],d=0;d<a;++d){var e=b[d].i(),f=this.resolve(e);if(f&&f!=this.j[e])try{b[d].tb(f)}catch(g){}else c.push(b[d])}this.i=c.concat(b.slice(a))};var Dd=function(a){_.T.call(this);this.o=a;this.A=this.i=null;this.v=0;this.B={};this.j=!1;a=window.navigator.userAgent;a.indexOf("MSIE")>=0&&a.indexOf("Trident")>=0&&(a=/\b(?:MSIE|rv)[: ]([^\);]+)(\)|;)/.exec(a))&&a[1]&&parseFloat(a[1])<9&&(this.j=!0)};_.A(Dd,_.T);Dd.prototype.C=function(a,b){this.i=b;this.A=a;b.preventDefault?b.preventDefault():b.returnValue=!1};_.Ed=function(a){this.J=_.w(a)};_.A(_.Ed,_.S);var Fd=function(a){this.J=_.w(a)};_.A(Fd,_.S);var Hd=function(){var a=Gd;this.i=null;_.M(a,4,!0)};Hd.prototype.log=function(a,b,c){c=c===void 0?new _.zd:c;_.Id(this,a,98,c)};_.Id=function(a,b,c,d){c=c===void
metadata
{
"source":"Luat.txt"
}

```

## Sau khi tiến hành chunking lại thì phát hiện thêm: ??? bug ở CHUNKING: mãi do TOKEN HUGGINGFACE của ông kia sai => FIX xong cái thì database lên Qdant và work ngon